In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
#import seaborn as sns

import keras_tuner
import tensorflow as tf
import tensorflow.keras.layers as tfl

from sklearn.metrics import confusion_matrix, classification_report

2023-01-14 10:30:30.192472: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
pd.options.display.max_seq_items = 2000
pd.options.display.max_columns = 500
pd.options.display.max_rows = 800
pd.options.display.max_colwidth = 200
pd.options.display.width = 800

In [4]:
base_url = '/Users/francesco/REPOS/name-clf/app'
os.chdir(base_url)

In [5]:
from models.hypermodel import hyperband_tuner, model

### Load artifacts

In [6]:
with np.load(os.path.join(base_url, "artifacts/data", "Country-datasets.npz"), allow_pickle=True) as f:
    X_train, y_train = f["X_train"], f["y_train"]
    X_val, y_val = f["X_val"], f["y_val"]
    
print(X_train.shape)
print(X_val.shape)

(7999, 32)
(1001, 32)


In [7]:
with open(os.path.join(base_url, "artifacts", "tokenizer.pkl"), "rb") as f:
    tokenizer = pickle.load(f)

### Build model

In [8]:
# Params from data
embedding_size = len(tokenizer.word_index)
print("embedding_size = ", embedding_size)
metrics = [tf.keras.metrics.CategoricalAccuracy()]
print("metrics = ", metrics)
softmax_units = y_train.shape[1]
print("softmax_units = ", softmax_units)
input_shape = X_train.shape[1]
print("input_shape = ", input_shape)

embedding_size =  68
metrics =  [<keras.metrics.metrics.CategoricalAccuracy object at 0x7fa8ae438700>]
softmax_units =  5
input_shape =  32


2023-01-14 10:30:40.872205: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
# Batch data
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

### Single model

In [10]:
my_model = model(input_shape, metrics, softmax_units, embedding_size)

32


In [11]:
my_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 32, 68)            4692      
                                                                 
 conv1d (Conv1D)             (None, 26, 32)            15264     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 13, 32)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 13, 32)            0         
                                                                 
 lstm (LSTM)                 (None, 13, 32)            8320      
                                                                 
 flatten (Flatten)           (None, 416)               0         
                                                        

In [12]:
history=my_model.fit(
    X_train, 
    y_train, 
    epochs=30, 
    batch_size=64,
    verbose =1,
    validation_data=(X_val,y_val)
)

Epoch 1/30
125/125 [==============================] - 8s 32ms/step - loss: 1.5211 - categorical_accuracy: 0.2922 - val_loss: 1.4829 - val_categorical_accuracy: 0.2737
Epoch 2/30
125/125 [==============================] - 4s 30ms/step - loss: 1.4473 - categorical_accuracy: 0.3468 - val_loss: 1.4447 - val_categorical_accuracy: 0.3546
Epoch 3/30
125/125 [==============================] - 3s 26ms/step - loss: 1.4188 - categorical_accuracy: 0.3610 - val_loss: 1.4248 - val_categorical_accuracy: 0.3756
Epoch 4/30
125/125 [==============================] - 4s 29ms/step - loss: 1.3938 - categorical_accuracy: 0.3874 - val_loss: 1.3764 - val_categorical_accuracy: 0.3896
Epoch 5/30
125/125 [==============================] - 2s 16ms/step - loss: 1.3780 - categorical_accuracy: 0.4002 - val_loss: 1.3913 - val_categorical_accuracy: 0.3906
Epoch 6/30
125/125 [==============================] - 3s 21ms/step - loss: 1.3752 - categorical_accuracy: 0.3975 - val_loss: 1.3812 - val_categorical_accuracy: 0.381

### Hyperband with Keras Tuner

In [13]:
project_name = "tuning_country"
directory = "logs/country_hpt"
tuner = hyperband_tuner(input_shape, metrics, softmax_units, embedding_size, project_name, directory)

In [14]:
tuner.search_space_summary(extended=True)

Search space summary
Default search space size: 5
dropout_1 (Float)
{'default': 0.3, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_units_1 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
dropout_2 (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': None}
dense_units_2 (Choice)
{'default': 64, 'conditions': [], 'values': [64, 128, 256], 'ordered': True}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


### Tuning

In [15]:
hpt_dir = "logs/tb_country_hpt"
fit_dir = "logs/tb_country_fit"
print(hpt_dir)
tensorboard_callback_htp = tf.keras.callbacks.TensorBoard(log_dir=hpt_dir, histogram_freq=1)
tensorboard_callback_fit = tf.keras.callbacks.TensorBoard(log_dir=fit_dir, histogram_freq=1)

logs/tb_country_hpt


In [16]:
%load_ext tensorboard

In [17]:
# Estimated trials simulation:
sim_max_epochs = 20
sim_iterations = 1
sim_factor = 3
est_trials = sim_iterations*sim_max_epochs*(math.log(sim_max_epochs, sim_factor))**2
print("Estimated trials: ", est_trials)

Estimated trials:  148.71236723665453


In [18]:
tuner.search(
    train_dataset,
    validation_data=val_dataset,
    callbacks=[tensorboard_callback_htp, tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)]
)

Trial 30 Complete [00h 01m 06s]
val_loss: 1.4114619493484497

Best val_loss So Far: 1.3295183181762695
Total elapsed time: 00h 12m 57s
INFO:tensorflow:Oracle triggered exit


In [19]:
tuner.results_summary(3)

Results summary
Results in logs/country_hpt/tuning_country
Showing 3 best trials
Trial summary
Hyperparameters:
dropout_1: 0.1
dense_units_1: 128
dropout_2: 0.5
dense_units_2: 256
lr: 0.0007487917450008943
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0020
Score: 1.3295183181762695
Trial summary
Hyperparameters:
dropout_1: 0.2
dense_units_1: 64
dropout_2: 0.0
dense_units_2: 128
lr: 0.0015790884158590982
tuner/epochs: 20
tuner/initial_epoch: 7
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0013
Score: 1.3298264741897583
Trial summary
Hyperparameters:
dropout_1: 0.30000000000000004
dense_units_1: 128
dropout_2: 0.30000000000000004
dense_units_2: 64
lr: 0.0004427191803516009
tuner/epochs: 20
tuner/initial_epoch: 0
tuner/bracket: 0
tuner/round: 0
Score: 1.3452163934707642


In [20]:
%tensorboard --logdir $hpt_dir

In [21]:
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
print(best_model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 32, 68)            4692      
                                                                 
 conv1d_1 (Conv1D)           (None, 26, 16)            7632      
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 16)           0         
 1D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 13, 16)            0         
                                                                 
 lstm_1 (LSTM)               (None, 13, 8)             800       
                                                                 
 flatten_1 (Flatten)         (None, 104)               0         
                                                      